In [ ]:
# ws01
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

cols = ['preg', 'plas', 'pres', 'skin', 'insu', 'bmi', 'pedi', 'age', 'class']
df = pd.read_csv("data/pima-indians-diabetes.csv", names=cols)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().round(2)

In [ ]:
df.hist(bins=20, figsize=(12, 6))
plt.show()

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
plt.boxplot(df.bmi)
plt.show()

In [ ]:
df[df.bmi == 0] 

In [ ]:
bmi_zero = df.bmi[df.bmi == 0].count() 
print('bmi zero = {} ' .format(bmi_zero))
print('{:.2f} %' .format(bmi_zero / df.shape[0] * 100))

In [ ]:
def chk_zero(cl):
    zero = df[cl][df[cl] == 0].count() 
    print('{:5} {:5} {:7.2f} %' .format(cl, zero, zero/df.shape[0]*100))

print('Zero count: \n--------------------')
for col in df.columns:
    chk_zero(col) 

In [ ]:
df['bmi'] = df.bmi.replace(0, df.bmi.mean())
# df['insu'] = df.insu.replace(0, 131) 
# df = df[df.bmi > 0]
# df = df[df.plas > 0]
# df = df[df.skin > 0]
# df = df[df.test > 0]
# df = df[(df.bmi > 0) & (df.plas > 0) & (df.skin > 0) & (df.insu > 0)]

In [ ]:
df.drop('insu', axis=1, inplace=True)

In [ ]:
df.describe().round(2)

In [ ]:

cor = df.corr()


g = sns.heatmap(cor, annot=True, cmap='YlGnBu', linewidths=0.8, 
                linecolor='b', fmt='.2f') 

g.set_yticklabels(g.get_yticklabels(), rotation=0, fontsize=12)
plt.show()

In [ ]:

cols_fts = ['preg','plas','bmi','pedi','age'] 

In [ ]:
X = df[cols_fts].copy()
X['test'] = df['age'] * .7 - df['pedi'] * 9
X.head()


In [ ]:

corr = X.corr()

g = sns.heatmap(corr, annot=True, cmap='YlGnBu', linewidths=0.8, 
                linecolor='y', fmt='.2f') 

g.set_yticklabels(g.get_yticklabels(), rotation=0, fontsize=12)
plt.show()

In [ ]:

X2 = X.copy()
X2['class'] = df['class']
corr = X2.corr()


g = sns.heatmap(corr, annot=True, cmap='YlGnBu', linewidths=0.8, 
                linecolor='w', fmt='.2f') 

g.set_yticklabels(g.get_yticklabels(), rotation=0, fontsize=12)
plt.show()

### ws02

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler as Scaler

X = df.drop('class', axis=1)

sc = Scaler() 
X_sc = sc.fit_transform(X)

df_sc = pd.DataFrame(X_sc, columns=X.columns)
df_sc.head().round(3)

In [ ]:
# variance
df_sc.var().sort_values(ascending=False).round(5) * 1000

### ws03

In [ ]:
df["class"] = df["class"].astype('category')
y = df['class']

In [ ]:
X = df.drop(['class'], axis=1)
X.head(3)

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler as Scaler

sc = Scaler() # scaler.fit(X)
X_sc = sc.fit_transform(X)
df_sc = pd.DataFrame(X_sc, columns=X.columns)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(X_sc, y)

np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(X_sc)

print(features[0:3,:])

In [ ]:
dfts = pd.DataFrame(fit.scores_ , columns=['score'])
dfts['column'] = X.columns
dfts.sort_values(by='score', ascending=False)

In [ ]:
cols_fts = dfts.sort_values(by='score', ascending=False).head(5).column
cols_fts.values

In [ ]:
cols_fts = dfts.nlargest(5, 'score').column
cols_fts.values

In [ ]:
X_fts = df_sc[cols_fts].copy()
X_fts.shape
X_fts.head()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

model = SVC(kernel='linear') # , probability=True) 76.7

cvs = cross_val_score(model, X_fts, y, cv=10)  # no need to use model.fit  
print('cross val scores {}'.format(cvs.round(2)))
print('Average (%) = {:.2f}' .format(cvs.mean() * 100 ))

### ws04

In [ ]:

from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC

model = LinearSVC()

rfe = RFE(model, 5) 
rfe = rfe.fit(X_sc, y)

print(rfe.support_)
print(rfe.ranking_)

In [ ]:
cols_fts = []
for i,cl in enumerate(X.columns):  
    if rfe.support_[i] == True:
        cols_fts.append(cl)
cols_fts

In [ ]:
X_fts = df_sc[cols_fts]
X_fts.shape
X_fts.head()